<a href="https://colab.research.google.com/github/aadi-kanwar/Neural-Network-Deep-Learning/blob/main/Exp_12_Implement_a_basic_CNN_(VGG16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Required Libraries and Dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

## Set Parameters

In [2]:
img_height, img_width = 224, 224
batch_size = 32
num_classes = 2     # Cats and Dogs

## Load the cats abd dogs dataset

In [3]:
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_file = tf.keras.utils.get_file("cat_and_dogs_filtered.zip", dataset_url, extract=True)
base_dir = os.path.join(os.path.dirname(zip_file), 'cats_and_dogs_filtered')

68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


## Set path for train and validation directories

In [4]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

## Load the pre-trained VGG16 model without the top layer

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


## Freeze the layers in the base model to retain pre-trained weights

In [6]:
for layer in base_model.layers:
    layer.trainable = False

## Add custom layers on top of the base model for our specific classification task

In [7]:
x = base_model.output
x = Flatten()(x)   # Flatten the output to 1D
x = Dense(512, activation='relu')(x)    # Add a dense layer with 512 units
x = Dropout(0.5)(x)     # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

## Combine base models and custom layers

In [8]:
model = Model(inputs = base_model.input, outputs = predictions)

## Compile the model

In [9]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Prepare the data with ImageDataGenerator for data augumentation

In [10]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Train the model

In [14]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size
)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 30s 445ms/step - accuracy: 0.8646 - loss: 0.2867 - val_accuracy: 0.9173 - val_loss: 0.1963
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8750 - loss: 0.3869 - val_accuracy: 0.8750 - val_loss: 0.1884
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 30s 428ms/step - accuracy: 0.8560 - loss: 0.3251 - val_accuracy: 0.9083 - val_loss: 0.2127
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9062 - loss: 0.2456 - val_accuracy: 0.8750 - val_loss: 0.1339
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 40s 449ms/step - accuracy: 0.8827 - loss: 0.2701 - val_accuracy: 0.9093 - val_loss: 0.2230
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7500 - loss: 0.4487 - val_accuracy: 0.7500 - val_loss: 0.3405
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 30s 456ms/step - accuracy: 0.8553 - loss: 0.2969 - val_accuracy: 0.9294 - val_loss: 0.1811
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8750 - loss: 0.2588 - val_accuracy: 1.00

## Evaluate the model

In [12]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - accuracy: 0.9387 - loss: 0.1759
Validation Loss: 0.2086
Validation Accuracy: 0.9260


## Save the model

In [13]:
model.save =("VGG16_transfer_learning_cats_dogs.h5")